# example_upload

```
#!/usr/bin/env python3
import requests

# This example shows how a file can be uploaded using
# The Python Requests module

url = "http://localhost/upload/"
with open('/usr/share/apache2/icons/icon.sheet.png', 'rb') as opened:
    r = requests.post(url, files={'file': opened})
```

# Code changeImage.py 

```
#!/usr/bin/env python3

import os
from PIL import Image
 
# path = os.path.expanduser('~') + '/supplier-data/images/'
path = 'supplier-data/images/'
 
for file in os.listdir(path):
    if '.tiff' in file:
        
        """
        The raw images from images subdirectory contains alpha transparency layers. 
        So, it is better to first convert RGBA 4-channel format to RGB 3-channel format before processing the images. 
        Use convert("RGB") method for converting RGBA to RGB image.
        """
        
        img = Image.open(path + file).convert("RGB")
        dir, filename = os.path.split(file)
        filename = os.path.splitext(filename)[0] # get filename without extension
        
        """
        After processing the images, save them in the same path ~/supplier-data/images, with a JPEG extension.
        """
        
        img.resize((600, 400)).save(path + filename + '.jpeg' , 'jpeg')
```

# Code supplier_image_upload.py:
```
#!/usr/bin/env python3

import requests
import os

url = "http://localhost/upload/"
# image_path = os.path.expanduser('~') + '/supplier-data/images/'
image_path = 'supplier-data/images/'
jpeg_images = [image for image in os.listdir(image_path) if '.jpeg' in image]

for image in jpeg_images:
    with open(image_path+image, 'rb') as opened:
        r = requests.post(url, files={'file': opened})
```

# Code run.py 
```
#! /usr/bin/env python3
 
import os 
import requests
import re 

# desc_path = os.path.expanduser('~') + '/supplier-data/descriptions/' 
# image_path = os.path.expanduser('~') + '/supplier-data/images/'
desc_path = 'supplier-data/descriptions/' 
image_path = 'supplier-data/images/'

text_files = sorted(os.listdir(desc_path))
jpeg_images = sorted([image_name for image_name in os.listdir(image_path) if '.jpeg' in image_name])
# print(text_files)
# print(jpeg_images)

list_content = []
image_counter = 0

for file in text_files:
    """
    Note that all files are written in the following format, with each piece of information on its own line:
    * name
    * weight (in lbs)
    * description
    """
    format = ['name', 'weight', 'description']
    
    with open(desc_path + file, 'r') as f:
        data = {}
        contents = f.read().split("\n")[0:3]  # read only first 3 line, because there is empty lines in the end of file
        # print(contents)

        """
        The weight field is defined as an integer field. 
        So when you process the weight information of the fruit from the .txt file, you need to convert it into an integer. 
        For example if the weight is "500 lbs", you need to drop "lbs" and convert "500" to an integer.
        """
        contents[1] = int((re.search(r'\d+', contents[1])).group()) # The weight field is defined as an integer field.
        # print(type(contents[1]))

        counter = 0
        for content in contents:
            # print(content)
            data[format[counter]] = content
            counter += 1

        """
        The image_name field will allow the system to find the image associated with the fruit. 
        Don't forget to add all fields, including the image_name!
        """
        data['image_name'] = jpeg_images[image_counter]

        list_content.append(data)
        image_counter += 1

"""
 The final JSON object should be similar to:
{
    "name": "Watermelon", 
    "weight": 500, 
    "description": "Watermelon is good for relieving heat, eliminating annoyance and quenching thirst. 
                    It contains a lot of water, which is good for relieving the symptoms of acute fever immediately. 
                    The sugar and salt contained in watermelon can diuretic and eliminate kidney inflammation. 
                    Watermelon also contains substances that can lower blood pressure.", 
    "image_name": "010.jpeg"
}
"""
# print(list_content) 

for item in list_content:
    resp = requests.post('http://127.0.0.1:80/fruits/', json=item)
    if resp.status_code != 201: 
        raise Exception('POST error status={}'.format(resp.status_code))
    print('Created feedback ID: {}'.format(resp.json()["id"]))
```


# reports.py 
```
#!/usr/bin/env python3

# Note: You can use and modify script from module 3
 
from reportlab.platypus import SimpleDocTemplate
from reportlab.platypus import Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
 
def generate_report(filename, title, additional_info):
  styles = getSampleStyleSheet()
  report = SimpleDocTemplate(filename)
  report_title = Paragraph(title, styles["h1"])
  report_info = Paragraph(additional_info, styles["Normal"])
  empty_line = Spacer(1,20)
  report.build([report_title, empty_line, report_info])
```

# Code report_email.py 
```
#!/usr/bin/env python3

import reports
import emails
import os 
from datetime import date
 
desc_path = os.path.expanduser('~') + '/supplier-data/descriptions/'
report = []
 
def process_data(data):
    for item in data:
        report.append("name: {}<br/>weight: {}\n".format(item[0], item[1]))
    return report
 
text_data = []
for text_file in os.listdir(desc_path):
    with open(desc_path + text_file, 'r') as f:
        text_data.append([line.strip() for line in f.readlines()])
        f.close()
 
if __name__ == "__main__":
    """
    You will need to pass the following arguments to the reports.generate_report method: 
    * the text description processed from the text files as the paragraph argument, 
    * the report title as the title argument, 
    * and the file path of the PDF to be generated as the attachment argument (use ‘/tmp/processed.pdf')
    """
    summary = process_data(text_data)
    paragraph = "<br/><br/>".join(summary)
    title = "Processed Update on {}\n".format(date.today().strftime("%B %d, %Y"))
    attachment = "/tmp/processed.pdf"
    reports.generate_report(attachment, title, paragraph)

    # Send the email
    """
    Use the following details to pass the parameters to emails.generate_email():
    * From: automation@example.com
    * To: username@example.com
        * Replace username with the username given in the Connection Details Panel on the right hand side.
    * Subject line: Upload Completed - Online Fruit Store
    * E-mail Body: All fruits are uploaded to our website successfully. A detailed list is attached to this email.
    * Attachment: Attach the path to the file processed.pdf
    """
    subject = "Upload Completed - Online Fruit Store"
    sender = "automation@example.com"
    receiver = "{}@example.com".format(os.environ.get('USER'))
    body = "All fruits are uploaded to our website successfully. A detailed list is attached to this email."
    message = emails.generate_email(sender, receiver, subject, body, attachment)
    emails.send_email(message)
```

# Code emails.py
```
#!/usr/bin/env python3

# Note: You can use and modify script from module 3

import email.message
import mimetypes
import os.path
import smtplib

def generate_email(sender, recipient, subject, body, attachment_path):
  """Creates an email with an attachement."""
  # Basic Email formatting
  message = email.message.EmailMessage()
  message["From"] = sender
  message["To"] = recipient
  message["Subject"] = subject
  message.set_content(body)

  # Process the attachment and add it to the email
  attachment_filename = os.path.basename(attachment_path)
  mime_type, _ = mimetypes.guess_type(attachment_path)
  mime_type, mime_subtype = mime_type.split('/', 1)

  with open(attachment_path, 'rb') as ap:
    message.add_attachment(ap.read(),
                          maintype=mime_type,
                          subtype=mime_subtype,
                          filename=attachment_filename)

  return message

def send_email(message):
  """Sends the message to the configured SMTP server."""
  mail_server = smtplib.SMTP('localhost')
  mail_server.send_message(message)
  mail_server.quit()


"""
Note: There is no attachment file here, so you must be careful while defining the generate_email() method in the emails.py script or you can create a separate generate_error_report() method for handling non-attachment email.
"""
def generate_error_report(sender, recipient, subject, body):
  message = email.message.EmailMessage()
  message["From"] = sender
  message["To"] = recipient
  message["Subject"] = subject
  message.set_content(body)
 
  return message
```

# Code health_check.py 
```
#! /usr/bin/env python3

"""
This Python script should send an email if there are problems, such as:
* Report an error if CPU usage is over 80%
* Report an error if available disk space is lower than 20%
* Report an error if available memory is less than 500MB
* Report an error if the hostname "localhost" cannot be resolved to "127.0.0.1"
"""

import os
import shutil
import psutil
import socket
from emails import generate_error_report, send_email
 
def check_cpu_usage():
    """Verifies that there's enough unused CPU"""
    usage = psutil.cpu_percent(1)
    # return usage > 80
    return usage < 80
 
def check_disk_usage(disk):
    """Verifies that there's enough free space on disk"""
    du = shutil.disk_usage(disk)
    free = du.free / du.total * 100
    return free > 20
 
def check_available_memory():
    """available memory in linux-instance, in byte"""
    available_memory = psutil.virtual_memory().available/(1024*1024)
    return available_memory > 500
 
def check_localhost():
    """check localhost is correctly configured on 127.0.0.1"""
    localhost = socket.gethostbyname('localhost')
    return localhost == '127.0.0.1'
 
to_be_checked = {
    check_cpu_usage(): "CPU usage is over 80%",
    check_disk_usage("/"): "Available disk space is less than 20%",
    check_available_memory(): "Available memory is less than 500MB",
    check_localhost(): "localhost cannot be resolved to 127.0.0.1"
}

error = False
for action, message in to_be_checked.items():
    if not action: 
        error_message = message
        error = True
# print(error_message)
 
# Send email
if error:
    try:
        sender = "automation@example.com"
        receiver = "{}@example.com".format(os.environ.get('USER'))
        subject = "Error - {}".format(error_message)
        body = "Please check your system and resolve the issue as soon as possible"
        message = generate_error_report(sender, receiver, subject, body)
        send_email(message)
        print('test')
    except NameError:
        pass
```